In [ ]:
import h5py 
import sys, warnings

import numpy as np 
import numpy.matlib as matlib 

import matplotlib
import matplotlib.pyplot as plt 
import matplotlib.animation as animation

from extra_data import RunDirectory 

from scipy import stats 
from IPython.display import clear_output  

import cv2 

import logging 
logging.captureWarnings(True) 

from sklearn.mixture import GaussianMixture 

# Setting up a random generator 
def_rng = np.random.default_rng() 

<h2> Input processing </h2> 

In [ ]:
saveText = bool(int(input(prompt='Save regression results: Y/N (1/0):')))  
selectMotor =  bool(int(input(prompt='Color using positions (1) or train ID (0):'))) 
if selectMotor:
    motorXY = bool(int(input(prompt='Motor X (1) or motor Y (0):'))) 
plotLog = bool(int(input(prompt='Plot in log-scale (1) or not (0):'))) 
plotCorr = bool(int(input(prompt='Plot correlation (1) or not (0):'))) 
plotMotorPulse = bool(int(input(prompt='Plot positions (1) or not (0) per pulse:'))) 
plotMotorTrain = bool(int(input(prompt='Plot positions (1) or not (0) per train:'))) 
saveVideo = bool(int(input(prompt='Save video of the Zyla microscope (1) or not (0):'))) 

<h2> Run(s) to correlate </h2> 

In [ ]:
run_list = [184,188,189,190,191,193,194,195,196] # liquid sheet - isopropanol (197 weird run) 

for r in [193,194]: 
    clear_output(wait=False) 
    sys.stderr.write(f'Processing run {r}...\n') 
    run = RunDirectory(path=f'/gpfs/exfel/d/raw/SPB/202202/p003046/r0{r}') 

    trainIDs_array = np.array(run.train_ids) 
    

    # XGM source(s) 
    xgm_src1 = 'SA1_XTD2_XGM/XGM/DOOCS:output' # before attenuator 
    xgm_src2 = 'SPB_XTD9_XGM/XGM/DOOCS:output' # after attenuator 

    xgm_src = xgm_src2 

    # Motor source(s) 
    motor_src = 'SPB_IRU_MOTORS/MDL/DATA_SELECT' 
    
    # Zyla source 
    zyla_src = 'SPB_EXP_ZYLA/CAM/1:daqOutput'

    # Loading motor positions 
    motor_x = np.array(run.get_array(motor_src,'SPB_IRU_INJMOV_MOTOR_X.actualPosition.value')) 
    motor_y = np.array(run.get_array(motor_src,'SPB_IRU_INJMOV_MOTOR_Y.actualPosition.value')) 
    
    # Loading Zyla images
    zyla = np.array(run.get_array(zyla_src,'data.image.pixels')) 
    zyla_tids = np.array(run.get_array(zyla_src,'data.trainId')) 
    zyla_x = np.array(run.get_array(motor_src,'SPB_IRU_INJMOV_MOTOR_X.actualPosition.value')) 
    zyla_y = np.array(run.get_array(motor_src,'SPB_IRU_INJMOV_MOTOR_Y.actualPosition.value')) 
    
    # Loading radial integrals 
    directory = '/gpfs/exfel/u/scratch/SPB/202202/p003046/data' 

    with h5py.File(directory+'/r0'+f'{r}'+'_proc_radavg.h5') as rad: 
        radavg = rad['entry_1']['radialavg'][:]
        trainIds = rad['entry_1']['trainId'][:]

        q = rad['entry_1']['q'][:] 

    xgm = np.array(run.get_array(xgm_src,'data.intensitySa1TD')) 
    agipd_tids = trainIds 
    xgm_tids = np.array(run.get_array(xgm_src,'data.trainId')) 

    # Print mismatch between AGIPD and XGM signals 
    unique_agipd = np.unique(agipd_tids) 
    unique_xgm = np.unique(xgm_tids) 
    
    # Unique Zyla Train IDs
    unique_zyla = np.unique(zyla_tids) 

    mismatch = np.fromiter(set(unique_agipd) - set(unique_xgm),int) 
    print(f'Pulse trains missing in XGM data: {mismatch}') 
    print(f'{mismatch.shape[0]} train(s) is/are missing in the XGM data') 

    # Train ID mask 
    good_train_mask = np.ones(shape=(trainIDs_array.shape[0],)) 
    for train in range(good_train_mask.shape[0]): 
        if trainIDs_array[train] in mismatch:
            good_train_mask[train]=0 

    # Remove data from AGIPD not in XGM data 
    good_pulse_mask = np.ones(shape=(radavg.shape[0],)) 
    for pulse in range(good_pulse_mask.shape[0]): 
        if agipd_tids[pulse] in mismatch: 
            good_pulse_mask[pulse] = 0 

    # Selecting the q-range to look at 
    num_pulses = 202 # maximum number of pulses 
    integrateQ = True # integrate the radial average over a certain q-range 

    q_sel = 80  
    q_max = 180 

    if integrateQ: 
        q_sel = np.arange(q_sel,q_max) 
        print(f'Integrating between {q[q_sel.min()]}-{q[q_sel.max()]} 1/nm') 
    else:
        print(f'Integrating at {q[q_sel]} 1/nm') 

    # Mask out data in AGIPD radial average and train IDs not matching XGM
    agipd = radavg[good_pulse_mask==1]
    agipd_tids = agipd_tids[good_pulse_mask==1] 

    pulse_tids = trainIds[good_pulse_mask==1] # use to color the pulses 

    # Mask out data where train not matched between XGM and AGIPD 
    motor_x = motor_x[good_train_mask==1] 
    motor_y = motor_y[good_train_mask==1] 
    
    # Copying the motor array to 
    motor_x_trains = motor_x.copy()
    motor_y_trains = motor_y.copy()

    # Expanding dimensions before padding with dummy pulses
    motor_x = np.expand_dims(motor_x,axis=1)
    motor_y = np.expand_dims(motor_y,axis=1)

    # Padding dummy pulses with motor positions and flatten 
    motor_x = matlib.repmat(motor_x,m=1,n=num_pulses).flatten() 
    motor_y = matlib.repmat(motor_y,m=1,n=num_pulses).flatten() 

    # Multi-train correlations 
    agipd_pulses, xgm_pulses = [], [] 
    agipd_per_train, xgm_per_train = [], [] 

    n_good, n_bad = 0, 0 
    hasnoPulse, hasPulse = [], [] 

    n_pulses = np.zeros(shape=(xgm.shape[0],),dtype=int) 
    pulse_tid = []

    mot_x, mot_y = [],[]

    for t in range(xgm.shape[0]): 

        # Ignore all the first pulses, select one part of the radial average for now 
        if integrateQ: 
            agipd_train = agipd[t*num_pulses:(1+t)*num_pulses][1:,q_sel].sum(axis=1) 
        else:
            agipd_train = agipd[t*num_pulses:(1+t)*num_pulses][1:,q_sel] 

        tid_train = pulse_tids[t*num_pulses:(1+t)*num_pulses][1:]
        mx_train = motor_x[t*num_pulses:(1+t)*num_pulses][1:]
        my_train = motor_y[t*num_pulses:(1+t)*num_pulses][1:]
    
        sel = xgm[t]>1. 
        xgm_train = xgm[t][sel] 
        if(xgm_train.shape[0] > 176): 
            agipd_train = agipd_train[:176]
            xgm_train = xgm_train[:176] 
            tid_train = tid_train[:176] 
            mx_train,my_train = mx_train[:176],my_train[:176] 
            
        #max_shape = xgm_train.shape[0] 
        #n_look_min, n_look_max = 0,max_shape # by default n_look_min=0 and n_look_max=xgm_train.shape[0] 
        #agipd_train = agipd_train[n_look_min:n_look_max] 
        #xgm_train = xgm_train[n_look_min:n_look_max] 
        #tid_train = tid_train[n_look_min:n_look_max] 
        #mx_train,my_train = mx_train[n_look_min:n_look_max],my_train[n_look_min:n_look_max] 

        if xgm_train.shape[0] == 0: 
            n_bad+=1 
            hasnoPulse.append(t) 
            continue  
        else:  
            step = 176//xgm_train.shape[0] 
            agipd_train = agipd_train[::step] 
            
            n_good+=1 
            hasPulse.append(t) 
            n_pulses[t] = xgm_train.shape[0] 

            # Put train IDs 
            id_sel = tid_train[:xgm_train.shape[0]]
            pulse_tid.extend(id_sel.flatten())

            # Put motor positions 
            mx_sel,my_sel = mx_train[:xgm_train.shape[0]],my_train[:xgm_train.shape[0]]
            mot_x.extend(mx_sel.flatten()),mot_y.extend(my_sel.flatten())

            # Select pulses present in both XGM and radial averages
            agipd_sel = agipd_train[:xgm_train.shape[0]] 
            xgm_sel = xgm_train[:xgm_train.shape[0]] 

            agipd_pulses.extend(agipd_sel.flatten()) 
            xgm_pulses.extend(xgm_sel.flatten()) 

            agipd_per_train.append(list(agipd_sel)) 
            xgm_per_train.append(list(xgm_sel)) 

    # Saving all pulses stacked together 
    agipd_pulses = np.array(agipd_pulses) 
    xgm_pulses = np.array(xgm_pulses)  
    pulse_tid = np.array(pulse_tid) 
    mot_x = np.array(mot_x) 
    mot_y = np.array(mot_y) 

    # Saving all pulses per train separately 
    agipd_per_train = np.array(agipd_per_train)
    xgm_per_train = np.array(xgm_per_train) 

    # Saving trains with/without pulses 
    hasPulse = np.array(hasPulse) 
    hasnoPulse = np.array(hasnoPulse) 

    print(f'{n_good} trains left for correlation analysis') 
    print(f'{n_bad} trains removed from correlation analysis') 
    print(f'Number of pulses after removal: {agipd_pulses.shape[0]}') 
    print(f'Trains have these numbers of pulses: {np.unique(n_pulses)}')   
    
    # Plotting AGIPD/XGM signals for single-train 
    checkSignal = False 
    t = 500 
    n_p = n_pulses[t] 
    if False: 
        # Plotting selected train for XGM and AGIPD 
        fig_handle = plt.figure(1,constrained_layout = True,dpi=100) 
        fig_handle.patch.set_facecolor(f'white') 
        spec_handle = fig_handle.add_gridspec(nrows = 2, ncols = 2) 

        ax_i = fig_handle.add_subplot(spec_handle[0,:2]) 
        im_i = plt.plot(agipd_pulses[t*n_p:(1+t)*n_p],'b')  
        ax_i.set_xlim([0,n_p])
        ax_i.set_xticks([0,n_p],minor=True) 
        ax_i.set_title(f'Radial signal - (train {t})',fontsize=7) 
        ax_i.set_xlabel('Pulse #') 

        ax_i = fig_handle.add_subplot(spec_handle[1,:2]) 
        im_i = plt.plot(xgm_pulses[t*n_p:(1+t)*n_p],'r') 
        ax_i.set_xlim([0,n_p]) 
        ax_i.set_xticks([0,n_p],minor=True)
        ax_i.set_title(f'XGM signal - (train {t})',fontsize=7) 
        ax_i.set_xlabel('Pulse #'); 

    # Single-train correlation 
    sel_agipd = agipd_pulses[t*n_p:(1+t)*n_p] 
    sel_xgm = xgm_pulses[t*n_p:(1+t)*n_p] 

    fit_1 = stats.linregress(sel_xgm, sel_agipd) 

    # Multi-train correlation 
    min_train,max_train = 0,100 
    offset_0, offset_1 = 0,0
    agipd_sel, xgm_sel,id_sel = [], [], []  
    mot_x_sel, mot_y_sel = [], [] 

    for tr in range(min_train,max_train):  
        n_pt = n_pulses[hasPulse][tr]
        agipd_sel.extend(agipd_pulses[tr*n_pt-offset_0:(1+tr)*n_pt+offset_1])  
        xgm_sel.extend(xgm_pulses[tr*n_pt-offset_0:(1+tr)*n_pt+offset_1])  
        id_sel.extend(pulse_tid[tr*n_pt-offset_0:(1+tr)*n_pt+offset_1]) 
        mot_x_sel.extend(mot_x[tr*n_pt-offset_0:(1+tr)*n_pt+offset_1]) 
        mot_y_sel.extend(mot_y[tr*n_pt-offset_0:(1+tr)*n_pt+offset_1]) 
        
    # Linear regression on nan-removed AGIPD/XGM pulses 
    agipd_sel = np.array(agipd_sel) 
    xgm_sel = np.array(xgm_sel) 
    id_sel = np.array(id_sel) 
    mot_x_sel = np.array(mot_x_sel)
    mot_y_sel = np.array(mot_y_sel) 

    # Check if XGM/AGIPD contains NaNs or Infs 
    # isfinite returns True if element is good and False is NaN or +/-Inf, so we invert the result to check for NaNs
    agipdCheck = ~np.isfinite(agipd_sel) 
    xgmCheck = ~np.isfinite(xgm_sel) 

    if agipdCheck.any(): 
        print('Only AGIPD signal has NaNs!') 
        agipdNaNs = np.squeeze(np.argwhere(agipdCheck==True)) # find pulses with NaNs
        agipdNoNaNs = np.squeeze(np.argwhere(agipdCheck==False)) # find pulses without NaNs
        print(f'There are {agipdNaNs.shape[0]} pulses with NaNs!') 
        agipd_sel = agipd_sel[agipdNoNaNs] 
        xgm_sel = xgm_sel[agipdNoNaNs] 
        id_sel = id_sel[agipdNoNaNs]
        mot_x_sel = mot_x_sel[agipdNoNaNs] 
        mot_y_sel = mot_y_sel[agipdNoNaNs]
    elif xgmCheck.any(): 
        print('Only XGM signal has NaNs!') 
        xgmNaNs = np.squeeze(np.argwhere(xgmCheck==True)) # find pulses with NaNs
        xgmdNoNaNs = np.squeeze(np.argwhere(xgmCheck==False)) # find pulses without NaNs
        print(f'There are {xgmNaNs.shape[0]} pulses with NaNs!') 
        xgm_sel = xgm_sel[xgmdNoNaNs] 
        agipd_sel = agipd_sel[xgmdNoNaNs] 
        id_sel = id_sel[xgmdNoNaNs]
        mot_x_sel = mot_x_sel[xgmdNoNaNs]
        mot_y_sel = mot_y_sel[xgmdNoNaNs] 
    elif agipdCheck.any() and xgmCheck.any(): 
        # TODO - fix code when both AGIPD/XGM have NaNs-especially if they occur at different locations 
        print('Both AGIPD and XGM have NaNs!') 
        print('Not yet implemented')
    else: 
        print('No NaNs present!') 

    # Performing linear fit 
    fit_2 = stats.linregress(xgm_sel,agipd_sel) 

    # Saving the linear regression to a text file 
    if saveText: 
        with open(f'linear_regression/lin_fit_run_{r}_trains_{min_train}-{max_train}.txt','w') as handle: 
            handle.write(f'number of trains used: {max_train-min_train}\n') 
            handle.write(f'fitting results\n') 
            handle.write(f'slope: {fit_2.slope}\n') 
            handle.write(f'intercept: {fit_2.intercept}\n') 
            handle.write(f'R^2: {fit_2.rvalue**2}\n') 
            handle.write(f'R: {fit_2.rvalue}\n') 
            handle.write(f'pvalue: {fit_2.pvalue}\n') 
            handle.write(f'stderr: {fit_2.stderr}\n') 
            handle.write(f'intercept_stderr: {fit_2.intercept_stderr}\n')

    # Select a way to color points 
    if selectMotor: 
        if motorXY: 
            c_points = mot_x_sel # uses motor X to color individual pulses 
            label = 'X' 
            mot_pos = np.unique(c_points) 
            motor_sel = motor_x_trains
        else: 
            c_points = mot_y_sel # uses motor Y to color individual pulses 
            label = 'Y' 
            mot_pos = np.unique(c_points)
            motor_sel = motor_y_trains
        fmt = '%.3f' 
    else: 
        c_points = id_sel # uses train ID to color individual pulses 
        label = 'trainID'
        fmt = '%.0f' 

    # Log-scale plotting 
    if plotLog:
        scale = 'log' # log,symlog 
    else:
        scale = 'linear'

    if plotCorr:
        # Plotting the scatter plots 
        fig_handle = plt.figure(2,constrained_layout = True,figsize=(10,10),dpi=100) 
        fig_handle.patch.set_facecolor(f'white') 
        spec_handle = fig_handle.add_gridspec(nrows = 1, ncols = 2) 
        
        fig_handle.suptitle(f'Correlation run {r}',x=0.5,y=1.0,weight='bold',size='8'); 
        
        # Plotting selected single-train AGIPD/XGM correlation 
        ax_i = fig_handle.add_subplot(spec_handle[0,0]) 
        im_i = plt.scatter(sel_xgm,sel_agipd,s=5,c='b',marker='o') 
        ax_i.plot(sel_xgm, (fit_1.slope * sel_xgm + fit_1.intercept) , "r", linewidth = 2) 
        ax_i.set_title(f'(train {t})',fontsize=6,weight='bold') 
        ax_i.set_xlabel('XGM',weight='bold') 
        ax_i.set_ylabel('AGIPD',weight='bold') 
        ax_i.annotate("$R^2$= " + str("%0.5f" % fit_1.rvalue**2), xy = (0.05, 0.90), 
                      xycoords = "axes fraction", weight = "bold", size = 8); 

        # Plotting selected multi-train AGIPD/XGM correlation 
        ax_i = fig_handle.add_subplot(spec_handle[0,1]) 
        im_i = plt.scatter(xgm_sel,agipd_sel,s=5,c=c_points,cmap='jet',marker='o',alpha=0.5) 
        if plotLog == False: 
            ax_i.plot(xgm_sel, (fit_2.slope * xgm_sel + fit_2.intercept) , "r", linewidth = 1) 
            ax_i.annotate("$R^2$= " + str("%0.5f" % fit_2.rvalue**2), xy = (0.05, 0.90), 
                      xycoords = "axes fraction", weight = "bold", size = 8)
        ax_i.set_title(f'({max_train-min_train} trains)',fontsize=6,weight='bold') 
        ax_i.set_xscale(scale)
        ax_i.set_yscale(scale)
        ax_i.set_xlabel('XGM',weight='bold') 
        ax_i.set_ylabel('AGIPD',weight='bold') 
        c_bar_i = plt.colorbar(im_i,ax=ax_i,ticklocation = 'top',orientation='horizontal',format=fmt)
        c_bar_i.set_ticks([c_points.min(),c_points.max()]) 
        c_bar_i.set_label(label,fontsize=6,weight='bold');
        
        plt.savefig(f'plots/new_axes/run_{r}_correlation_{label}_trains_{min_train}_{max_train}.pdf');  
        plt.close(); 
        
    if plotMotorPulse: 
        # Calculating clusters of pulses based on unique motor positions (depends on the scan direction) with linear fit 
        n_c = mot_pos.shape[0] 

        # Fitting linear regression line to each unique motor position
        slope_c = np.zeros(shape=(n_c,)) 
        intercept_c =np.zeros(shape=(n_c)) 
        rsq_c = np.zeros(shape=(n_c,)) 

        for c in range(n_c): 
            fit_n = stats.linregress(xgm_sel[np.squeeze(np.argwhere(c_points==mot_pos[c]))],
                                     agipd_sel[np.squeeze(np.argwhere(c_points==mot_pos[c]))]) 
            slope_c[c],intercept_c[c],rsq_c[c] = fit_n.slope,fit_n.intercept,fit_n.rvalue**2 

        # Number of pulses per unique motor position 
        n_points_c = np.zeros(shape=(n_c,)) 
        for n in range(n_c): 
            n_points_c[n] = (c_points == mot_pos[n]).astype(int).sum() 

        fig_handle = plt.figure(3,constrained_layout=True,figsize=(15,15));  
        fig_handle.patch.set_facecolor(f'white'); 
        spec_handle = fig_handle.add_gridspec(nrows=4, ncols=2);

        fig_handle.suptitle(f'Run {r} motor {label} - pulse resolved',x=0.5,y=1.0,weight='bold',size='15'); 
        fig_handle.set_size_inches(w=15.00, h=20.00, forward=True) 
        
        ### 0 
        ax_i = fig_handle.add_subplot(spec_handle[0,0]) 
        ax_i.plot(mot_pos,rsq_c, "ro-", linewidth=1) 
        ax_i.set_title('$R^{2}$ versus motor position',fontsize=10,weight='bold') 
        ax_i.set_xlabel('motor '+label,weight='bold') 
        ax_i.set_ylabel('$R^{2}$',weight='bold'); 
        #
        ax_i = fig_handle.add_subplot(spec_handle[0,1]) 
        ax_i.hist(rsq_c,bins=50) 
        ax_i.set_xlabel('$R^{2}$',weight='bold')  
        ax_i.set_ylabel('count',weight='bold');
        ### 1 
        ax_i = fig_handle.add_subplot(spec_handle[1,0]) 
        ax_i.plot(mot_pos,intercept_c, "go-", linewidth=1) 
        ax_i.set_title('Intercept versus motor position',fontsize=10,weight='bold') 
        ax_i.set_xlabel('motor '+label,weight='bold') 
        ax_i.set_ylabel('Intercept',weight='bold'); 
        #
        ax_i = fig_handle.add_subplot(spec_handle[1,1]) 
        ax_i.hist(intercept_c,bins=50) 
        ax_i.set_xlabel('Intercept',weight='bold') 
        ax_i.set_ylabel('count',weight='bold'); 
        ### 2
        ax_i = fig_handle.add_subplot(spec_handle[2,0]) 
        ax_i.plot(mot_pos,slope_c, "bo-", linewidth=1) 
        ax_i.set_title('Slope versus motor position',fontsize=10,weight='bold') 
        ax_i.set_xlabel('motor '+label,weight='bold') 
        ax_i.set_ylabel('Slope',weight='bold'); 
        # 
        ax_i = fig_handle.add_subplot(spec_handle[2,1]) 
        ax_i.hist(slope_c,bins=50) 
        ax_i.set_xlabel('Slope',weight='bold') 
        ax_i.set_ylabel('count',weight='bold');
        ### 3
        ax_i = fig_handle.add_subplot(spec_handle[3,0:2]) 
        ax_i.plot(mot_pos,n_points_c,"mo-",linewidth=1)
        ax_i.set_xlabel('motor '+label,weight='bold') 
        ax_i.set_ylabel('Number of pulses',weight='bold'); 
        
        plt.savefig(f'plots/new_axes/run_{r}_motor_{label}_trains_{min_train}_{max_train}_pulse.pdf');  
        plt.close(); 
        
    if plotMotorTrain: 
        # Random subset of trains 
        cmap = np.array(matplotlib.cm.get_cmap('Paired').colors) # any qualitative colormap 
        cmap = np.expand_dims(cmap,axis=0) 
        
        n_c = mot_pos.shape[0] 
        n_t = cmap.shape[1] 

        # Fitting linear regression line to each unique motor position
        slope_c = np.zeros(shape=(n_t,)) 
        intercept_c =np.zeros(shape=(n_t,)) 
        rsq_c = np.zeros(shape=(n_t,)) 
        
        # Storing (randomly) selected trains
        ag_rand = [] 
        xg_rand = [] 
        c_rand = [] 
        
        trains = def_rng.choice(n_good,size=n_t, replace=False) 
        
        # Fitting linear regression line within each train separately 
        for t in range(n_t): 
            t_l = trains[t]
            agCheck = ~np.isfinite(agipd_per_train[t_l]) 
            xgCheck = ~np.isfinite(xgm_per_train[t_l]) 

            if agCheck.any() == True:
                agNaNs = np.squeeze(np.argwhere(agCheck==True)) 
                agNoNaNs = np.squeeze(np.argwhere(agCheck==False)) 
                ag_t = np.array(agipd_per_train[t_l])[agNoNaNs]
                xg_t = np.array(xgm_per_train[t_l])[agNoNaNs] 
            elif xgCheck.any() == True:
                xgNaNs = np.squeeze(np.argwhere(xgCheck==True)) 
                xgNoNaNs = np.squeeze(np.argwhere(xgCheck==False)) 
                ag_t = np.array(agipd_per_train[t_l])[xgNoNaNs] 
                xg_t = np.array(xgm_per_train[t_l])[xgNoNaNs] 
            elif agCheck.any() and xgCheck.any(): 
                print('NaNs present in both AGIPD and XGM!') 
            else:
                ag_t = np.array(agipd_per_train[t_l])
                xg_t = np.array(xgm_per_train[t_l]) 
                
            fit_n = stats.linregress(xg_t,ag_t) 
            slope_c[t],intercept_c[t],rsq_c[t] = fit_n.slope,fit_n.intercept,fit_n.rvalue**2
            ag_rand.append(ag_t), xg_rand.append(xg_t)
        
            ct = np.expand_dims(t_l ,axis=(0,1)) 
            ct = matlib.repmat(ct,m=1,n= n_pulses[t_l]).T
            c_rand.append(ct)
            
        # Fitting linear regression line within each train separately 
        slope_all = np.zeros(shape=(n_good,)) 
        intercept_all =np.zeros(shape=(n_good,)) 
        rsq_all = np.zeros(shape=(n_good,)) 
        for t_l in range(n_good): 
            agCheck = ~np.isfinite(agipd_per_train[t_l]) 
            xgCheck = ~np.isfinite(xgm_per_train[t_l]) 

            if agCheck.any() == True:
                agNaNs = np.squeeze(np.argwhere(agCheck==True)) 
                agNoNaNs = np.squeeze(np.argwhere(agCheck==False)) 
                ag_t = np.array(agipd_per_train[t_l])[agNoNaNs]
                xg_t = np.array(xgm_per_train[t_l])[agNoNaNs] 
            elif xgCheck.any() == True:
                xgNaNs = np.squeeze(np.argwhere(xgCheck==True)) 
                xgNoNaNs = np.squeeze(np.argwhere(xgCheck==False)) 
                ag_t = np.array(agipd_per_train[t_l])[xgNoNaNs] 
                xg_t = np.array(xgm_per_train[t_l])[xgNoNaNs] 
            elif agCheck.any() and xgCheck.any(): 
                print('NaNs present in both AGIPD and XGM!') 
            else:
                ag_t = np.array(agipd_per_train[t_l])
                xg_t = np.array(xgm_per_train[t_l]) 

            fit_n = stats.linregress(xg_t,ag_t) 
            slope_all[t_l],intercept_all[t_l],rsq_all[t_l] = fit_n.slope,fit_n.intercept,fit_n.rvalue**2 
            
        # Converting lists to arrays
        ag_rand,xg_rand,c_rand = np.squeeze(np.array(ag_rand)),np.squeeze(np.array(xg_rand)),np.squeeze(np.array(c_rand))
        
        # Number of trains per unique motor position 
        n_points_c = np.zeros(shape=(n_c,)) 
        for c in range(n_c): 
            n_points_c[c] = (motor_sel[trains] == mot_pos[c]).astype(int).sum() 

        fig_handle = plt.figure(4,constrained_layout=True,figsize=(15,15));  
        fig_handle.patch.set_facecolor(f'white'); 
        spec_handle = fig_handle.add_gridspec(nrows=4, ncols=2);

        fig_handle.suptitle(f'Run {r} motor {label} - train ({n_t}/{n_good} trains) resolved',x=0.55,y=1.0,weight='bold',size='15'); 
        fig_handle.set_size_inches(w=15.00, h=20.00, forward=True) 

        train_sel = np.unique(id_sel) - id_sel.min()
        fmt = '%.0d'
        
        ### 0 
        ax_i = fig_handle.add_subplot(spec_handle[0,0]) 
        im_i=ax_i.scatter(motor_y_trains,rsq_all, marker=".") 
        ax_i.set_title('$R^{2}$ versus motor position',fontsize=10,weight='bold') 
        ax_i.set_xlabel('motor '+label,weight='bold') 
        ax_i.set_ylabel('$R^{2}$',weight='bold'); 
        
        ax_i = fig_handle.add_subplot(spec_handle[0,0]) 
        im_i=ax_i.scatter(motor_y_trains[trains],rsq_c, marker="o", c=cmap[0]) 
        ax_i.set_title('$R^{2}$ versus motor position',fontsize=10,weight='bold') 
        ax_i.set_xlabel('motor '+label,weight='bold') 
        ax_i.set_ylabel('$R^{2}$',weight='bold'); 
        #
        ax_i = fig_handle.add_subplot(spec_handle[0,1])
        for t in range(n_t):
            im_i=ax_i.scatter(xg_rand[t],ag_rand[t],c=cmap[:,t]) 
            ax_i.plot(xg_rand[t], (slope_c[t] * xg_rand[t] + intercept_c[t]),c=np.squeeze(cmap[:,t]),linestyle="--") 
        ax_i.set_xlabel('XGM',weight='bold') 
        ax_i.set_ylabel('AGIPD',weight='bold') 
        ### 1 
        ax_i = fig_handle.add_subplot(spec_handle[1,0]) 
        im_i=ax_i.scatter(motor_y_trains,intercept_all,marker= ".") 
        ax_i.set_title('Intercept versus motor position',fontsize=10,weight='bold') 
        ax_i.set_xlabel('motor '+label,weight='bold') 
        ax_i.set_ylabel('Intercept',weight='bold'); 
        
        ax_i = fig_handle.add_subplot(spec_handle[1,0]) 
        im_i=ax_i.scatter(motor_y_trains[trains],intercept_c,marker= "o",c=cmap[0]) 
        ax_i.set_title('Intercept versus motor position',fontsize=10,weight='bold') 
        ax_i.set_xlabel('motor '+label,weight='bold') 
        ax_i.set_ylabel('Intercept',weight='bold'); 
        #
        ax_i = fig_handle.add_subplot(spec_handle[1,1]) 
        for t in range(n_t):
            im_i=ax_i.scatter(xg_rand[t],ag_rand[t],c=cmap[:,t]) 
            ax_i.plot(xg_rand[t], (slope_c[t] * xg_rand[t] + intercept_c[t]),c=np.squeeze(cmap[:,t]),linestyle="--") 
        ax_i.set_xlabel('XGM',weight='bold') 
        ax_i.set_ylabel('AGIPD',weight='bold') 
        ### 2
        ax_i = fig_handle.add_subplot(spec_handle[2,0]) 
        im_i=ax_i.scatter(motor_y_trains,slope_all,marker= ".") 
        ax_i.set_title('Slope versus motor position',fontsize=10,weight='bold') 
        ax_i.set_xlabel('motor '+label,weight='bold') 
        ax_i.set_ylabel('Slope',weight='bold')
        
        ax_i = fig_handle.add_subplot(spec_handle[2,0]) 
        im_i=ax_i.scatter(motor_y_trains[trains],slope_c,marker= "o", c=cmap[0]) 
        ax_i.set_title('Slope versus motor position',fontsize=10,weight='bold') 
        ax_i.set_xlabel('motor '+label,weight='bold') 
        ax_i.set_ylabel('Slope',weight='bold')
        # 
        ax_i = fig_handle.add_subplot(spec_handle[2,1]) 
        for t in range(n_t):
            im_i=ax_i.scatter(xg_rand[t],ag_rand[t],c=cmap[:,t]) 
            ax_i.plot(xg_rand[t], (slope_c[t] * xg_rand[t] + intercept_c[t]),c=np.squeeze(cmap[:,t]),linestyle="--") 
        ax_i.set_xlabel('XGM',weight='bold') 
        ax_i.set_ylabel('AGIPD',weight='bold') 
        ### 3
        ax_i = fig_handle.add_subplot(spec_handle[3,0:2]) 
        ax_i.plot(mot_pos,n_points_c,"mo--",linewidth=1) 
        ax_i.set_xlabel('motor '+label,weight='bold') 
        ax_i.set_ylabel('Number of trains',weight='bold'); 
        
        plt.savefig(f'plots/new_axes/run_{r}_motor_{label}_train.pdf');  
        plt.close();

    sys.stderr.write(f'Finished processing run {r}...\n') 
    
if saveVideo: 
    res = 150  
    figure = plt.figure(dpi=res) 
    n_frames = zyla.shape[0] 
    frame = 0 
    image = plt.imshow(zyla[frame],vmin=0,interpolation=None ) 
    plt.xticks([]) 
    plt.yticks([]) 
    plt.colorbar(shrink=0.7) 

    def update_frame(*args): 
        global frame 

        image.set_data(zyla[frame]) 
        plt.title(f'Run {r}: train ({zyla_tids[frame]-zyla_tids.min()}/{zyla_tids.shape[0]})',weight='bold',size='14') 
        figure.suptitle(f'Motor: ({zyla_x[frame]:.10f}, {zyla_y[frame]:.10f})',x=0.53,y=0.08, 
                     weight='bold',size='14'); 

        frame += 1 
        frame %= n_frames 

        return image, 
    
    sys.stderr.write('Saving animation...\n') 
    ani = animation.FuncAnimation(figure,update_frame,n_frames,interval=1,repeat=True,blit=False) 
    ani_writer = animation.ImageMagickFileWriter(fps=60,bitrate=-1) 
    ani.save(f'gifs/run_{r}_{res}_dpi.gif',writer=ani_writer) 
    sys.stderr.write('Finished saving animation...') 

In [ ]:
res = 150  
figure = plt.figure(1,dpi=res) 
n_frames = zyla.shape[0] 
frame = 0 
zyla_frame = zyla[frame]
image = plt.imshow(zyla_frame,vmin=0,interpolation=None) 
plt.xticks([]) 
plt.yticks([]) 
plt.colorbar(shrink=0.7); 

frame_blur = cv2.GaussianBlur(zyla_frame, (3,3), 0);
figure = plt.figure(2,dpi=res) 
image = plt.imshow(frame_blur,vmin=0,interpolation=None) 

<h2> Exploring various data sources and keys </h2> 

In [ ]:
src_list = ['SPB_IRU_LIQUIDJET/FLOW/BRONKHORST_1','SPB_IRU_LIQUIDJET/FLOW/BRONKHORST_SAMPLE1',
'SPB_IRU_LIQUIDJET/FLOW/BRONKHORST_SAMPLE2','SPB_IRU_LIQUIDJET/FLOW/BRONKHORST_WASH1',
'SPB_IRU_LIQUIDJET/FLOW/BRONKHORST_WASH2','SPB_IRU_LIQUIDJET/MDL/MANIFOLD_VALVES',
'SPB_IRU_LIQUIDJET/VALVE/VALVE_SAMPLE_1','SPB_IRU_LIQUIDJET/VALVE/VALVE_SAMPLE_2',
'SPB_IRU_LIQUIDJET/VALVE/VALVE_SAMPLE_3','SPB_IRU_LIQUIDJET/VALVE/VALVE_SAMPLE_4'] 

src = src_list[5]
run.keys_for_source(src)

k =  'SPB_IRU_LIQUIDJETVALVEVALVE_SAMPLE_2.nPositions.value'
carr = np.array(run.get_array(src,k)) 

plt.figure(1,dpi=100)
plt.plot(carr)
plt.title(f'{k}'); 

<h2> Gaussian mixture model clustering of pulses (WIP) </h2> 

In [ ]:
# Plotting clusters of pulses based on motor positions (depends on the scan direction) with linear fit 
n_c = 5 
if n_c % 2 !=0:
    offset=1 
else: 
    offset = 0 
    
# Keep on fitting pulses to GMM until all classes have at least one pulse 
gmm = GaussianMixture(n_components=n_c,covariance_type='full',init_params='random') 
data_stack = np.vstack([agipd_sel,xgm_sel]).T 

while True: 
    c_fit = gmm.fit(data_stack) 
    c_pred = c_fit.predict(data_stack) 
    if np.unique(c_pred).shape[0] == n_c:
        break 
    else:
        continue 

# Fitting linear regression line to each cluster  
slope_c = np.zeros(shape=(n_c,)) 
intercept_c =np.zeros(shape=(n_c)) 
rsq_c = np.zeros(shape=(n_c,)) 

for c in range(n_c): 
    fit_n = stats.linregress(agipd_sel[np.squeeze(np.argwhere(c_pred==c))], xgm_sel[np.squeeze(np.argwhere(c_pred==c))]) 
    slope_c[c],intercept_c[c],rsq_c[c] = fit_n.slope,fit_n.intercept,fit_n.rvalue**2 

# Plotting all pulse classes 
fig_handle = plt.figure(4,constrained_layout=True,figsize=(n_c*(n_c//2+offset),n_c*2),dpi=200) 
fig_handle.patch.set_facecolor(f'white') 
spec_handle = fig_handle.add_gridspec(nrows = n_c//2+offset, ncols = 2) 
cc = 0
for r in range(spec_handle.get_geometry()[0]): 
    for c in range(spec_handle.get_geometry()[1]): 
        if cc==n_c:
            break 
        s_fit = np.squeeze(np.argwhere(c_pred==cc)) 
        ag = agipd_sel[s_fit] 
        xg = xgm_sel[s_fit] 

        ax_i = fig_handle.add_subplot(spec_handle[r,c]) 
        im_i = plt.scatter(ag,xg,s=0.1,c=c_points[s_fit],cmap='jet',marker='x',alpha=0.5) 
        ax_i.plot(ag, (slope_c[cc] * ag + intercept_c[cc]) , "r", linewidth = 1) 
        ax_i.set_title(f'Correlation {cc} ({s_fit.shape[0]} pulses)',fontsize=10,weight='bold') 
        ax_i.set_xlabel('AGIPD',weight='bold') 
        ax_i.set_ylabel('XGM',weight='bold') 
        ax_i.annotate("$R^2$= " + str("%0.5f" % rsq_c[cc]), xy = (0.05, 0.85), 
                  xycoords = "axes fraction", weight = "bold", size = 15) 
        c_bar_i = plt.colorbar(im_i,ax=ax_i,ticklocation = 'top',orientation='horizontal',format=fmt)
        c_bar_i.set_ticks([c_points[s_fit].min(),c_points[s_fit].max()]) 
        c_bar_i.set_label(label,fontsize=10,weight='bold'); 
        cc+=1 